In [1]:
# %load D:/BUDS_MID/__init__.py
"""
Created on Wed Feb 19 16:21:00 2020

@author: cmchico

usual imports
"""

import os
import pandas as pd
import numpy as np

pd.options.display.max_rows = 100
pd.set_option('display.float_format', '{:.0f}'.format)

import f_sched_branch as ed


from time import time
from datetime import timedelta

import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# %load D:/BUDS_MID/__initdask__.py
"""
Created on Wed Feb 19 16:24:03 2020

@author: cmchico
"""
import dask.bag as db
from dask.distributed import Client, progress

client = Client(processes = True,n_workers=12)
client

'\nCreated on Wed Feb 19 16:24:03 2020\n\n@author: cmchico\n'

Client Scheduler: tcp://127.0.0.1:58941 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 12 Memory: 34.14 GB


##### parameters and data source

In [3]:
path = "D:/BUDS_MID/OJT ATM/ATM Forecasting Data New/"

In [4]:
test_dates = pd.date_range(start='01/01/2019',end='12/31/2019')

In [5]:
r_mapp = pd.read_pickle(path+'Actual/89 TID Br map.pkl')
r_thresh = pd.read_pickle(path+'89 Threshold Recommendations.pkl')
r_reload_df = pd.read_pickle(path+'Actual/80 Replenishment.pkl')
r_df = pd.concat([pd.read_csv(path+'Prophet Forecast/'+i, dtype={'ds':'str','pred_date':'str','tid':'str'}) 
                for i in os.listdir(path+'Prophet Forecast/')],ignore_index=True)
r_actual = pd.read_pickle(path+'Actual/82 Aggregated TID WDL.pkl')

##### count of atm per group (branch)

In [6]:
#211 branches with just one ATM, can be treated as an ATM i/o Branch
mapp = r_mapp.copy()
atm_count_dict = mapp[['TID_BR','BR_TIDCOUNT']].drop_duplicates().set_index('TID_BR').BR_TIDCOUNT.to_dict()
atm_br_dict = mapp.set_index('TID').TID_BR.to_dict()

In [7]:
mapp[['TID_BR','BR_TIDCOUNT']].drop_duplicates().BR_TIDCOUNT.value_counts()

1    514
2    128
3     14
4      2
Name: BR_TIDCOUNT, dtype: int64

##### threshold

In [8]:
r_thresh

,tid,pred_date,y,yhat,se,rmse,y_ave,thresh,y_ave_bucket
0,91010033,2018-12-30,1500,607252,366935198929,539692,775019,600000,800000
1,91010033,2018-12-31,14600,560825,298362048450,539692,775019,600000,800000
2,91010033,2019-01-01,142600,704290,315495965097,539692,775019,600000,800000
3,91010033,2019-01-02,318500,859928,293143790593,539692,775019,600000,800000
4,91010033,2019-01-03,325700,762721,190987445428,539692,775019,600000,800000
...,...,...,...,...,...,...,...,...,...
230953,BR998,2019-12-11,2753300,2665938,7632206157,533860,2228375,600000,2200000
230954,BR998,2019-12-12,1850600,2252653,161646243029,533860,2228375,600000,2200000
230955,BR998,2019-12-13,2890700,1908325,965060752512,533860,2228375,600000,2200000
230956,BR998,2019-12-14,3267700,2079174,1412594570874,533860,2228375,600000,2200000


In [9]:
thresh = r_thresh.copy()
thresh['per_rmse_yave'] =thresh.rmse/thresh.y_ave*100
thresh.tid = thresh.tid.astype(str)
thresh['tid_br'] = thresh.tid
thresh = thresh.sort_values(['per_rmse_yave','pred_date'], ascending=False)
thresh['tid_count'] = thresh['tid'].map(atm_count_dict)
# thresh.info()

In [10]:
thresh[thresh.tid_count > 1]

,tid,pred_date,y,yhat,se,rmse,y_ave,thresh,y_ave_bucket,per_rmse_yave,tid_br,tid_count
126359,BR155,2019-12-15,9317000,5636803,13543846569205,2415601,6541295,2500000,6500000,37,BR155,2
126358,BR155,2019-12-14,10228000,5187575,25405885150402,2415601,6541295,2500000,6500000,37,BR155,2
126357,BR155,2019-12-13,11411000,5439197,35662431044471,2415601,6541295,2500000,6500000,37,BR155,2
126356,BR155,2019-12-12,13275000,6917962,40411929628056,2415601,6541295,2500000,6500000,37,BR155,2
126355,BR155,2019-12-11,11600000,7420589,17467475031284,2415601,6541295,2500000,6500000,37,BR155,2
...,...,...,...,...,...,...,...,...,...,...,...,...
109165,BR029,2019-01-03,5834700,6909282,1154726010086,807045,6893774,900000,6900000,12,BR029,2
109164,BR029,2019-01-02,6324400,7357270,1066819916135,807045,6893774,900000,6900000,12,BR029,2
109163,BR029,2019-01-01,5864600,7244323,1903634949435,807045,6893774,900000,6900000,12,BR029,2
109162,BR029,2018-12-31,4304300,6634568,5430146775602,807045,6893774,900000,6900000,12,BR029,2


In [11]:
atms = thresh[thresh.tid_count > 1].tid_br.unique()
len(atms)

144

##### replenishment

In [12]:
reload_df = r_reload_df.copy()
reload_df = reload_df.sort_values(['TID','DATE'],ignore_index=True)
reload_df['Pre-Emptive'] = reload_df['reload_type'].str.strip().fillna("NORMAL LOADING")
reload_df['date_str'] = reload_df.DATE.astype(str)

reload_df['Current Load Amount'] = reload_df['Current Load Amount'].astype(str).str.strip().str.replace("","")
select = reload_df['Current Load Amount']==""
reload_df.loc[select,'Current Load Amount'] = "0"
reload_df['Current Load Amount'] = reload_df['Current Load Amount'].astype(int)

reload_df = reload_df[reload_df.DATE.isin(test_dates)].copy()
reload_df = reload_df.sort_values(['TID','DATE'],ignore_index=True)
reload_df['tid'] = reload_df.TID
reload_df['tid_br'] = reload_df.TID.map(atm_br_dict)
reload_df['tid_count'] = reload_df.tid_br.map(atm_count_dict)
# reload_df[['_forecasted','_cassettefault','_bpi','_others']] = \
#         (reload_df[['_forecasted','_cassettefault','_bpi','_others']]*1).fillna(0)


reload_df =  reload_df[reload_df.tid_br.isin(atms)].reset_index(drop=True)
reload_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48737 entries, 0 to 48736
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Last Load Amount     48737 non-null  object        
 1   Current Load Amount  48737 non-null  int32         
 2   % Cash Return/Atm    48737 non-null  float64       
 3   Replen               48737 non-null  object        
 4   Pre-Emptive          48737 non-null  object        
 5   _forecasted          48737 non-null  bool          
 6   _cassettefault       48737 non-null  bool          
 7   _bpi                 48737 non-null  bool          
 8   _others              48737 non-null  bool          
 9   reload_type          48737 non-null  object        
 10  TID                  48737 non-null  object        
 11  DATE                 48737 non-null  datetime64[ns]
 12  date_str             48737 non-null  object        
 13  tid                  48737 non-

In [13]:
# type_reload = np.array(['forecasted','cassettefault','bpi','others'])
# def get_type(x):
#     return '-'.join((type_reload[x.values]).tolist())

# reload_df['reload_type'] = reload_df[['_forecasted','_cassettefault','_bpi','_others']].fillna(False) \
#                                 .apply(get_type, axis=1)

# reload_df.groupby(['Pre-Emptive', 'reload_type']).size()
# # reload_df.reload_type.value_counts(normalize=True)*100

In [14]:
first_together = reload_df.groupby(['tid_br','tid_count','DATE']).size().reset_index()
first_together.columns = ['tid_br','tid_count','DATE','count_date']
first_together['check'] =first_together.tid_count == first_together.count_date
first_reload= first_together[first_together.check].sort_values(['DATE']).drop_duplicates(subset='tid_br')
first_reload.DATE.dt.month.value_counts()

1    142
4      1
2      1
Name: DATE, dtype: int64

In [15]:
reload_df

,Last Load Amount,Current Load Amount,% Cash Return/Atm,Replen,Pre-Emptive,_forecasted,_cassettefault,_bpi,_others,reload_type,TID,DATE,date_str,tid,tid_br,tid_count
0,5200000,5200000,30,LOADING,FORECAST,True,False,False,False,FORECAST,91010018,2019-01-01,2019-01-01,91010018,BR054,2
1,5200000,5200000,82,LOADING,FAILURE,True,True,False,False,FAILURE,91010018,2019-01-03,2019-01-03,91010018,BR054,2
2,5200000,5200000,2,LOADING,FORECAST,True,False,False,False,FORECAST,91010018,2019-01-06,2019-01-06,91010018,BR054,2
3,5200000,5200000,4,LOADING,FORECAST,True,False,False,False,FORECAST,91010018,2019-01-09,2019-01-09,91010018,BR054,2
4,5200000,5200000,15,LOADING,FAILURE,True,True,False,False,FAILURE,91010018,2019-01-12,2019-01-12,91010018,BR054,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48732,5000000,5200000,77,LOADING,FAILURE,False,True,False,False,FAILURE,91019983,2019-12-19,2019-12-19,91019983,BR691,2
48733,5200000,5000000,71,LOADING,FORECAST,False,False,False,True,FORECAST,91019983,2019-12-20,2019-12-20,91019983,BR691,2
48734,5000000,5000000,21,LOADING,FORECAST,True,False,False,False,FORECAST,91019983,2019-12-23,2019-12-23,91019983,BR691,2
48735,5000000,5000000,33,LOADING,FORECAST,True,False,False,False,FORECAST,91019983,2019-12-26,2019-12-26,91019983,BR691,2


In [16]:
preempt = reload_df[reload_df.reload_type != 'FORECAST'][['date_str','tid','tid_br','reload_type','Current Load Amount']] 
preempt

,date_str,tid,tid_br,reload_type,Current Load Amount
1,2019-01-03,91010018,BR054,FAILURE,5200000
4,2019-01-12,91010018,BR054,FAILURE,5200000
5,2019-01-15,91010018,BR054,FAILURE,5200000
6,2019-01-16,91010018,BR054,FAILURE,5200000
7,2019-01-18,91010018,BR054,FAILURE,5200000
...,...,...,...,...,...
48727,2019-12-11,91019983,BR691,FAILURE,5200000
48729,2019-12-13,91019983,BR691,FAILURE,5000000
48731,2019-12-18,91019983,BR691,FAILURE,5000000
48732,2019-12-19,91019983,BR691,FAILURE,5200000


##### actual withdrawal

In [17]:
wdl = r_actual[r_actual.DATE.isin(test_dates)].copy()
wdl['tid'] = wdl.TID.astype(str)
wdl['tid_br'] = wdl.tid.map(atm_br_dict)
wdl['date_str'] = wdl.DATE.astype(str)
wdl = wdl[wdl.tid_br.isin(atms)].reset_index(drop=True)
wdl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111690 entries, 0 to 111689
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   DATE         111690 non-null  datetime64[ns]
 1   TID          111690 non-null  int64         
 2   WDL_AMT      111690 non-null  float64       
 3   WDL_AMT_FIN  111690 non-null  float64       
 4   tid          111690 non-null  object        
 5   tid_br       111690 non-null  object        
 6   date_str     111690 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB


##### forecasts

In [18]:
df = r_df[r_df.tid.isin(atms)].copy()
df['tid_br'] = df.tid
df.sort_values(['tid','pred_date','ds'], inplace=True)
df['pred_date_D'] = pd.to_datetime(df.pred_date)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707616 entries, 1488942 to 3228497
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Unnamed: 0   707616 non-null  int64         
 1   ds           707616 non-null  object        
 2   y            707616 non-null  float64       
 3   yhat         707616 non-null  float64       
 4   pred_date    707616 non-null  object        
 5   pred_day     707616 non-null  object        
 6   se           707616 non-null  float64       
 7   tid          707616 non-null  object        
 8   tid_br       707616 non-null  object        
 9   pred_date_D  707616 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 59.4+ MB


In [19]:
actual = df.pivot_table(index=['tid_br','pred_date','pred_date_D'],columns='pred_day',values='yhat',aggfunc=sum).reset_index()
actual[['date_str','y']] = df[df.pred_day=='day 0'][['ds','y']].reset_index(drop=True)
actual = actual[actual.tid_br.isin(atms)].reset_index(drop=True)
actual.shape
actual.head()

(50544, 19)

pred_day,tid_br,pred_date,pred_date_D,day 0,day 1,day 10,day 11,day 12,day 13,day 2,day 3,day 4,day 5,day 6,day 7,day 8,day 9,date_str,y
0,BR003,2018-12-30,2018-12-30,3117300,2083878,3376936,3985918,885142,590059,3101453,3090027,4825599,640978,221006,2940485,2931412,2806236,2018-12-31,229800
1,BR003,2018-12-31,2018-12-31,765283,3084612,3969005,866640,569091,3361469,3070202,4805487,616784,198386,2917100,2914159,2795472,3356278,2019-01-01,50400
2,BR003,2019-01-01,2019-01-01,3084078,3079838,849397,553388,3367489,4729958,4796612,627296,197792,2926643,2887279,2765152,3340869,3949593,2019-01-02,2672400
3,BR003,2019-01-02,2019-01-02,3087783,4798795,565362,3380463,4726880,3170090,635664,206674,2937916,2896338,2777541,3343632,3965859,862642,2019-01-03,3395300
4,BR003,2019-01-03,2019-01-03,4807112,640374,3387445,4735531,3173275,3236330,208489,2944571,2901269,2779882,3356841,3967066,865222,566586,2019-01-04,4914900


In [20]:
#round-off forecasts to nearest 1000
actual[[x for x in actual.columns if 'day' in x]] = \
        (actual[[x for x in actual.columns if 'day' in x]] / 1000).astype(int)*1000

In [21]:
def data_prep(atm):
    atm_no = len(tid)
    date_o = first_reload[first_reload.tid_br == atm].reset_index().DATE[0]
    
    wdl_atm = wdl[wdl.tid_br==atm].pivot_table(index=['date_str','tid_br'], columns=['tid'],values=['WDL_AMT_FIN']).reset_index()
    wdl_atm.columns = wdl_atm.columns.get_level_values(0)[:2].tolist() + ('wdl_' + wdl_atm.columns.get_level_values(1)[2:]).tolist()


    actual_atm = actual[(actual.tid_br== atm) & (actual.pred_date_D >= date_o-timedelta(days=1))] \
                    [['tid_br','pred_date','date_str','y'] + window_col].reset_index(drop=True) \
                    .merge(wdl_atm,how='left')

    #in case more than 1 reload type per date, just get the first one to flag all atms
    g4s = preempt[preempt.tid_br==atm][['date_str','tid_br','reload_type']].drop_duplicates(subset=['date_str','tid_br']) \
        .merge(preempt[preempt.tid_br==atm][['date_str','tid_br','tid','Current Load Amount']])
    g4s = g4s.pivot_table(index=['date_str','tid_br','reload_type'],
                            columns=['tid'],values=['Current Load Amount']).reset_index().fillna(0)
    g4s.columns = g4s.columns.get_level_values(0)[:3].tolist() + ('c_'+g4s.columns.get_level_values(1)[3:]).tolist()
    
    return actual_atm, g4s

In [22]:
def perf(aa,reload_amt,atm_no,tid):
    
    ss =aa.groupby(['reload_type'])[['civ_cost','reload_cost','total_cost','cashout_flag','reload_flag']].sum().reset_index()
    reloads = {tid[i]+ str(i):reload_amt[i] for i in range(atm_no)}

    a=pd.Series(reloads).reset_index()
    a.columns=['tid','reload_amt']

    ss['tid_br']=atm
    ss['thresh'] = threshold
    ss['thresh%'] = thresh[thresh.tid_br==atm].per_rmse_yave.astype(int).unique()[0]

    avail = (aa[['avail_' +str(i) for i in range(atm_no)]].sum()/aa.shape[0]*100).astype(int).reset_index(drop=True)
    avail.name = 'Availability'

    ss=ss.join(a.join(avail),how='left')
    return ss

In [23]:
def par(reload_amt,i,actual_atm,atm, tid,g4s,t,threshold,r,reload_charge,days_no,hor):
    atm_no=len(tid)
    try:
        g4s[g4s.columns[3:]]= (g4s[g4s.columns[3:]]>0)*reload_amt
        aa = ed.sched_branch(actual_atm,reload_amt, tid,t,threshold,
                           r, reload_charge,days_no,hor,g4s)
    except:
        return [[atm,tid,reload_amt,'fail']]
    
    aa.to_csv(path+'Prophet Test Thresh R/'+'_'+atm+'_'+str(i)+'_.csv')
    ss = perf(aa,reload_amt,atm_no,tid)
    ss.to_csv(path+'Prophet Test Perf Thresh R/'+'_'+atm+'_'+str(i)+'_.csv')
    return [[atm,tid,reload_amt,'success']]
#     except:
        

def par_atm(atm):
    threshold = thresh[thresh.tid==atm].thresh.unique()[0]
    tid = wdl[wdl.tid_br== atm].tid.unique().tolist()
    atm_no=len(tid)    
    t = min(100000,int(threshold/hor/atm_no/100000)*100000)
    
    actual_atm, g4s = data_prep(atm)

    
    scenario = list(itertools.permutations(choice,atm_no))
    scenario_all = [list(i) for i in scenario] + [[a for i in range(atm_no)] for a in choice]
    
    print(len(scenario_all))
    error = [[atm,tid,scenario_all[i],"success"] for i in range(len(scenario_all))]
    
    for i in range(len(scenario_all)):
        reload_amt=scenario_all[i]
        try:
            par(reload_amt,i,actual_atm,atm, tid,g4s,t,threshold,r,reload_charge,days_no,hor)
            
#             g4s[g4s.columns[3:]]= (g4s[g4s.columns[3:]]>0)*reload_amt
#             aa=ed.sched_branch(actual_atm,reload_amt, tid,t,threshold,
#                                r, reload_charge,days_no,hor,g4s)
#             aa.to_pickle(path+'Prophet Test Thresh R/'+'_'+atm+'_'+str(i)+'_.pkl')
#             ss = perf(aa,atm_no,tid)
#             print(ss.total_cost.sum())
#             ss.to_csv(path+'Prophet Test Perf Thresh R/'+'_'+atm+'_'+str(i)+'_.csv')
        except:
            error[i] = [atm,tid,scenario_all[i],"fail"]
        print(timedelta(seconds=time()-start))
    return error

In [1]:

import itertools
choice=[3200000,4200000,5200000,6200000]


In [4]:
atm_no=2
scenario = list(itertools.permutations(choice,atm_no))
scenario_all = [list(i) for i in scenario] + [[a for i in range(atm_no)] for a in choice]
print(len(scenario_all))

atm_no=3
scenario = list(itertools.permutations(choice,atm_no))
scenario_all = [list(i) for i in scenario] + [[a for i in range(atm_no)] for a in choice]
print(len(scenario_all))
scenario_all
# atm_no=4
# scenario = list(itertools.permutations(choice,atm_no))
# scenario_all = [list(i) for i in scenario] + [[a for i in range(atm_no)] for a in choice]
# print(len(scenario_all))

16
28


[[3200000, 4200000, 5200000],
 [3200000, 4200000, 6200000],
 [3200000, 5200000, 4200000],
 [3200000, 5200000, 6200000],
 [3200000, 6200000, 4200000],
 [3200000, 6200000, 5200000],
 [4200000, 3200000, 5200000],
 [4200000, 3200000, 6200000],
 [4200000, 5200000, 3200000],
 [4200000, 5200000, 6200000],
 [4200000, 6200000, 3200000],
 [4200000, 6200000, 5200000],
 [5200000, 3200000, 4200000],
 [5200000, 3200000, 6200000],
 [5200000, 4200000, 3200000],
 [5200000, 4200000, 6200000],
 [5200000, 6200000, 3200000],
 [5200000, 6200000, 4200000],
 [6200000, 3200000, 4200000],
 [6200000, 3200000, 5200000],
 [6200000, 4200000, 3200000],
 [6200000, 4200000, 5200000],
 [6200000, 5200000, 3200000],
 [6200000, 5200000, 4200000],
 [3200000, 3200000, 3200000],
 [4200000, 4200000, 4200000],
 [5200000, 5200000, 5200000],
 [6200000, 6200000, 6200000]]

In [10]:
scenario_all

[[3200000, 4200000, 5200000, 6200000],
 [3200000, 4200000, 6200000, 5200000],
 [3200000, 5200000, 4200000, 6200000],
 [3200000, 5200000, 6200000, 4200000],
 [3200000, 6200000, 4200000, 5200000],
 [3200000, 6200000, 5200000, 4200000],
 [4200000, 3200000, 5200000, 6200000],
 [4200000, 3200000, 6200000, 5200000],
 [4200000, 5200000, 3200000, 6200000],
 [4200000, 5200000, 6200000, 3200000],
 [4200000, 6200000, 3200000, 5200000],
 [4200000, 6200000, 5200000, 3200000],
 [5200000, 3200000, 4200000, 6200000],
 [5200000, 3200000, 6200000, 4200000],
 [5200000, 4200000, 3200000, 6200000],
 [5200000, 4200000, 6200000, 3200000],
 [5200000, 6200000, 3200000, 4200000],
 [5200000, 6200000, 4200000, 3200000],
 [6200000, 3200000, 4200000, 5200000],
 [6200000, 3200000, 5200000, 4200000],
 [6200000, 4200000, 3200000, 5200000],
 [6200000, 4200000, 5200000, 3200000],
 [6200000, 5200000, 3200000, 4200000],
 [6200000, 5200000, 4200000, 3200000],
 [3200000, 3200000, 3200000, 3200000],
 [4200000, 4200000, 42000

##### start

In [24]:
hor=2
days_no = 7
r = 0.05/365
reload_charge=2000
window_col = ['day '+ str(i) for i in range(days_no+hor)]

In [25]:
import itertools

In [26]:
choice=[3200000,4200000,5200000,6200000]

In [27]:
scenario_ = pd.read_csv(path+'93 Scenarios.csv')
scen=dict()
scen[1] = [[x] for x in scenario_[scenario_.atm_no==1].rel1.values] 
scen[2] = scenario_[scenario_.atm_no==2][['rel1','rel2']].values.tolist()
scen[3] =  scenario_[scenario_.atm_no==3][['rel1','rel2','rel3']].values.tolist()
scen[4] = scenario_[scenario_.atm_no==4][['rel1','rel2','rel3','rel4']].values.tolist()

In [28]:
len(scen[4])

35

In [29]:
atm='BR003'
threshold = thresh[thresh.tid==atm].thresh.unique()[0]
tid = wdl[wdl.tid_br== atm].tid.unique().tolist()
atm_no=len(tid)    
t = min(100000,int(threshold/hor/atm_no/100000)*100000)

scenario_all=scen[atm_no]
print(len(scenario_all))

actual_atm, g4s = data_prep(atm)

10


In [30]:
# a=par(scenario_all[0],0,actual_atm=actual_atm,atm=atm, 
#            tid=tid,g4s=g4s,t=t,threshold=threshold,
#            r=r,reload_charge=reload_charge,days_no=days_no,hor=hor)

### sequential 2mins

In [31]:
start=time()
# for i in range(len(scenario_all)):
for i in range(1):
    reload_amt=scenario_all[i]
    g4s[g4s.columns[3:]]= (g4s[g4s.columns[3:]]>0)*reload_amt
    aa=ed.sched_branch(actual_atm,reload_amt, tid,t,threshold,
                       r, reload_charge,days_no,hor,g4s)
    aa.to_pickle(path+'Prophet Test Thresh R/'+'_'+atm+'_'+str(i)+'_.pkl')
    ss = perf(aa,reload_amt, atm_no,tid)
    print(ss.total_cost.sum())
    ss.to_csv(path+'Prophet Test Perf Thresh R/'+'_'+atm+'_'+str(i)+'_.csv')
    print(timedelta(seconds=time()-start))

cashout
cashout
cashout
cashout
cashout
cashout
cashout
cashout
cashout
cashout
cashout
cashout


TypeError: 'NoneType' object is not subscriptable

### 583 PROBLEMATIC

### parallel scenario, one atm 1:08secs

In [34]:
atm_new = list(set(atms)-set(['BR583']))

In [40]:
atm_new[59:]
atm_new.index('BR056')

['BR863',
 'BR373',
 'BR086',
 'BR981',
 'BR671',
 'BR985',
 'BR360',
 'BR986',
 'BR318',
 'BR881',
 'BR821',
 'BR186',
 'BR622',
 'BR600',
 'BR056',
 'BR895',
 'BR037',
 'BR885',
 'BR158',
 'BR666',
 'BR078',
 'BR180',
 'BR832',
 'BR871',
 'BR976',
 'BR419',
 'BR859',
 'BR883',
 'BR389',
 'BR461',
 'BR144',
 'BR626',
 'BR691',
 'BR873',
 'BR100',
 'BR602',
 'BR127',
 'BR810',
 'BR749',
 'BR347',
 'BR122',
 'BR397',
 'BR440',
 'BR057',
 'BR824',
 'BR581',
 'BR174',
 'BR377',
 'BR623',
 'BR098',
 'BR074',
 'BR667',
 'BR128',
 'BR807',
 'BR764',
 'BR846',
 'BR858',
 'BR159',
 'BR417',
 'BR503',
 'BR547',
 'BR199',
 'BR067',
 'BR058',
 'BR015',
 'BR055',
 'BR384',
 'BR605',
 'BR460',
 'BR559',
 'BR054',
 'BR507',
 'BR636',
 'BR569',
 'BR228',
 'BR971',
 'BR307',
 'BR878',
 'BR161',
 'BR808',
 'BR811',
 'BR564',
 'BR992',
 'BR374']

73

In [48]:
start = time()
for atm in newatm:
    print(atm)
    threshold = thresh[thresh.tid==atm].thresh.unique()[0]
    tid = wdl[wdl.tid_br== atm].tid.unique().tolist()
    atm_no=len(tid)    
    
    t = min(100000,int(threshold/hor/atm_no/100000)*100000)
    
    actual_atm, g4s = data_prep(atm)

    scenario = list(itertools.permutations(choice,atm_no))
    scenario_all = [list(i) for i in scenario] + [[a for ia in range(atm_no)] for a in choice]

    dbscen = db.from_sequence(scenario_all[i] for i in range(len(scenario_all)))
    dbi = db.from_sequence(i for i in range(len(scenario_all)))
    dbmaster = \
    db.map(par,dbscen,dbi,actual_atm=actual_atm,atm=atm, 
           tid=tid,g4s=g4s,t=t,threshold=threshold,
           r=r,reload_charge=reload_charge,days_no=days_no,hor=hor)
    a = dbmaster.compute()
    b = pd.concat([pd.DataFrame(a[i], columns=['atm','tid','reload_amt','status']) for i in range(len(scenario_all))]
          ,ignore_index=True)
    pd.DataFrame(b).to_pickle(path+'Prophet Forecast All/_'+atm+'_.pkl')
    print(b.status.value_counts())
    print(timedelta(seconds=time()-start))

BR996
success    10
fail        6
Name: status, dtype: int64
0:00:45.444874
BR346
success    16
Name: status, dtype: int64
0:01:39.558969
BR619
success    16
Name: status, dtype: int64
0:02:31.483947
BR332
success    16
Name: status, dtype: int64
0:03:22.310169
BR051
success    16
Name: status, dtype: int64
0:04:15.740079
BR045
success    15
fail        1
Name: status, dtype: int64
0:05:05.594220
BR610
success    16
Name: status, dtype: int64
0:05:56.219565
BR316
success    16
Name: status, dtype: int64
0:06:44.631066
BR088
success    28
Name: status, dtype: int64
0:10:42.644506
BR455
success    15
fail        1
Name: status, dtype: int64
0:11:34.213946
BR874
success    15
fail        1
Name: status, dtype: int64
0:12:27.312890
BR738
success    16
Name: status, dtype: int64
0:13:19.238872
BR418
success    14
fail        2
Name: status, dtype: int64
0:14:12.644684
BR853
success    16
Name: status, dtype: int64
0:15:05.053749
BR354
fail       10
success     6
Name: status, dtype: int64
0

In [54]:
for atm in ['BR347', 'BR377']:
    start = time()
    print(atm)
    threshold = thresh[thresh.tid==atm].thresh.unique()[0]
    tid = wdl[wdl.tid_br== atm].tid.unique().tolist()
    atm_no=len(tid)    
    
    t = min(100000,int(threshold/hor/atm_no/100000)*100000)
    
    actual_atm, g4s = data_prep(atm)

    scenario = list(itertools.permutations(choice,atm_no))
    scenario_all = [list(i) for i in scenario] + [[a for ia in range(atm_no)] for a in choice]

    dbscen = db.from_sequence(scenario_all[i] for i in range(len(scenario_all)))
    dbi = db.from_sequence(i for i in range(len(scenario_all)))
    dbmaster = \
    db.map(par,dbscen,dbi,actual_atm=actual_atm,atm=atm, 
           tid=tid,g4s=g4s,t=t,threshold=threshold,
           r=r,reload_charge=reload_charge,days_no=days_no,hor=hor)
    a = dbmaster.compute()
    b = pd.concat([pd.DataFrame(a[i], columns=['atm','tid','reload_amt','status']) for i in range(len(scenario_all))]
          ,ignore_index=True)
    pd.DataFrame(b).to_pickle(path+'Prophet Forecast All/_'+atm+'_.pkl')
    print(b.status.value_counts())
    print(timedelta(seconds=time()-start))

BR347
success    28
Name: status, dtype: int64
0:02:03.339702
BR377
success    16
Name: status, dtype: int64
0:00:49.735959


In [49]:
generated = pd.Series(os.listdir(path+'Prophet Test Thresh R/'))
# .str.split('_').str[1].unique()

In [50]:
generated = generated[generated.str.contains("csv")]
generated = generated.str.split('_').str[1].unique()

In [51]:
generated

array(['BR003', 'BR015', 'BR019', 'BR025', 'BR029', 'BR037', 'BR045',
       'BR051', 'BR054', 'BR055', 'BR056', 'BR057', 'BR058', 'BR067',
       'BR069', 'BR071', 'BR074', 'BR077', 'BR078', 'BR081', 'BR086',
       'BR087', 'BR088', 'BR091', 'BR098', 'BR100', 'BR121', 'BR122',
       'BR127', 'BR128', 'BR144', 'BR155', 'BR158', 'BR159', 'BR161',
       'BR163', 'BR174', 'BR178', 'BR180', 'BR186', 'BR191', 'BR199',
       'BR228', 'BR237', 'BR273', 'BR274', 'BR307', 'BR308', 'BR314',
       'BR316', 'BR318', 'BR319', 'BR331', 'BR332', 'BR339', 'BR346',
       'BR347', 'BR354', 'BR360', 'BR373', 'BR374', 'BR377', 'BR384',
       'BR389', 'BR397', 'BR405', 'BR409', 'BR412', 'BR414', 'BR417',
       'BR418', 'BR419', 'BR440', 'BR446', 'BR447', 'BR455', 'BR460',
       'BR461', 'BR485', 'BR494', 'BR503', 'BR507', 'BR547', 'BR557',
       'BR559', 'BR561', 'BR564', 'BR569', 'BR574', 'BR581', 'BR583',
       'BR600', 'BR602', 'BR605', 'BR610', 'BR619', 'BR622', 'BR623',
       'BR626', 'BR6

In [53]:
newatm = list(set(atms)-set(generated))
newatm

[]

In [46]:
os.listdir(path+'Prophet Test Thresh R/')

['BRANCH',
 'OFFSITE OR ONE ATM BRANCH',
 '_BR003_0_.csv',
 '_BR003_10_.csv',
 '_BR003_11_.csv',
 '_BR003_13_.csv',
 '_BR003_14_.csv',
 '_BR003_15_.csv',
 '_BR003_1_.csv',
 '_BR003_2_.csv',
 '_BR003_3_.csv',
 '_BR003_4_.csv',
 '_BR003_5_.csv',
 '_BR003_6_.csv',
 '_BR003_7_.csv',
 '_BR003_8_.csv',
 '_BR003_9_.csv',
 '_BR015_0_.csv',
 '_BR015_10_.csv',
 '_BR015_11_.csv',
 '_BR015_12_.csv',
 '_BR015_13_.csv',
 '_BR015_14_.csv',
 '_BR015_15_.csv',
 '_BR015_1_.csv',
 '_BR015_2_.csv',
 '_BR015_3_.csv',
 '_BR015_4_.csv',
 '_BR015_5_.csv',
 '_BR015_6_.csv',
 '_BR015_7_.csv',
 '_BR015_8_.csv',
 '_BR015_9_.csv',
 '_BR019_0_.csv',
 '_BR019_10_.csv',
 '_BR019_11_.csv',
 '_BR019_12_.csv',
 '_BR019_13_.csv',
 '_BR019_14_.csv',
 '_BR019_15_.csv',
 '_BR019_1_.csv',
 '_BR019_2_.csv',
 '_BR019_3_.csv',
 '_BR019_4_.csv',
 '_BR019_5_.csv',
 '_BR019_6_.csv',
 '_BR019_7_.csv',
 '_BR019_8_.csv',
 '_BR019_9_.csv',
 '_BR025_0_.csv',
 '_BR025_10_.csv',
 '_BR025_11_.csv',
 '_BR025_12_.csv',
 '_BR025_13_.csv',
 '

### parallel atm, sequential scenario

In [28]:
#per atm
start=time()
dbatm = db.from_sequence(atms)
dbmaster = db.map(par_atm,dbatm)
a=dbmaster.compute()
print(timedelta(seconds=time()-start))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
pd.concat([pd.DataFrame(a[i],columns=['tid','reload_amt','status']) for i in range(len(a))]).to_csv(path+'86 Summary Status.csv')